In [4]:
path = "C:/Users/RAZER/Desktop/TEMp/New folder/KINECT_dataset_with_qor15.csv"

In [1]:
path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"

In [2]:
# Re-import libraries and reload the dataset after code execution state reset
import pandas as pd

# Reload the dataset
file_path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"
df = pd.read_csv(file_path)

# Step 1: Filter data by a single walking speed
# Check unique walking speeds
unique_speeds = df['walking_speed'].unique()

# Assume we want to use 'Fast' walking speed
filtered_df = df[df['walking_speed'] == 'Fast'].copy()

# Get number of unique patients in this filtered set
unique_patients = filtered_df['patient_id'].nunique()

# Group by patient to prepare for next step
grouped_by_patient = filtered_df.groupby('patient_id')

unique_speeds, filtered_df.shape, unique_patients


(array(['Fast', 'Regular'], dtype=object), (42172, 113), 76)

In [8]:
import pandas as pd
import plotly.graph_objects as go

# Load dataset and filter
df = df[df['walking_speed'] == 'Fast'].copy()

# Define joints and edges
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'),
         ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

# Choose first patient and row
example_patient_id = df['patient_id'].unique()[0]
row = df[df['patient_id'] == example_patient_id].iloc[0]

# Extract 3D joint positions and normalize to pelvis
pelvis = [row['PELVIS_X'], row['PELVIS_Y'], row['PELVIS_Z']]
positions = {}
for j in joints:
    x, y, z = row[f"{j}_X"], row[f"{j}_Y"], row[f"{j}_Z"]
    positions[j] = (x - pelvis[0], y - pelvis[1], z - pelvis[2])

# Build edge lines
edge_traces = []
for j1, j2 in edges:
    x0, y0, z0 = positions[j1]
    x1, y1, z1 = positions[j2]
    edge_traces.append(go.Scatter3d(
        x=[x0, x1], y=[y0, y1], z=[z0, z1],
        mode='lines',
        line=dict(color='gray', width=4),
        hoverinfo='none'
    ))

# Build node markers
x_vals = [positions[j][0] for j in joints]
y_vals = [positions[j][1] for j in joints]
z_vals = [positions[j][2] for j in joints]
labels = joints

node_trace = go.Scatter3d(
    x=x_vals, y=y_vals, z=z_vals,
    mode='markers+text',
    marker=dict(size=5, color='red'),
    text=labels,
    textposition="top center"
)

# Final figure
fig = go.Figure(data=edge_traces + [node_trace])
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z'),
        aspectmode='data'
    ),
    title="3D Human Skeleton (Normalized to Pelvis)",
    showlegend=False
)

fig.show()
